## This Notebook will pull in and summarize Robot Scouter JSON Data.

The Notebook exports its results to CSV and Excel files located in a 'LatestResults' directory 
relative to this Jupyter Notebook.

** NO NEED TO RUN THE NEXT CODE SECTION IF YOU HAVE ALREADY INSTALLED THE FOLLOWING PYTHON LIBRARIES **

In [1]:
# NO NEED TO RUN THIS CODE SECTION IF YOU HAVE ALREADY INSTALLED THE FOLLOWING PYTHON LIBRARIES. 
# No problem though if you accidently run it as it will just say the "Requirement already satisfied"

!pip install requests
!pip install pandas

# The following library is currently being used for the Excel writer.  Was planning to use for exporting 
# to an Excel file format. Panda no longer supports exporting directly into an Excel file format.
!pip install openpyxl

In [2]:
import requests
import pandas as pd
from datetime import datetime

In [3]:
# Getting a unique file timestamp
currentTimeStamp = datetime.now()
file_timestamp = currentTimeStamp.strftime("%Y-%m-%d-T%H-%M")

In [4]:
# Reading in the MATCH_METRICS JSON file that was exported from Robot Scouter phone app
df_scouting = pd.read_json('LatestInputOfScoutData/MATCH_METRICS.json')

In [5]:
# Making it easier to designate the header / column name to use per metric collected.
# NOTE:  Multiple unique IDs for the same field since we had trouble with the template fields either disappearing
# or accidently getting deleted
d_labels = {'BTaK47cPx6fRaNgEyZUT': 'Scout', 'R0O96f6geNzPSJ9htvbw': 'Scout', 'g9Jp0qj9Jy01eJQDySvP': 'Scout',
            'aoFxSZtjIt9ivErdykgt': 'Scouted Match', 'X7h2ljauERqiKy28kS7e': 'Scouted Match', 
            'audn4HiEfE4mAc4H8sLd': 'AUTO Taxied', 'ghAL5qr5qHDayFAFqhM4': 'AUTO Taxied',
            'YaXQNGcVzGtDxPpy9wgA': 'AUTO Upper Attempts', 'd6duYpJtMZXIUulurS7G': 'AUTO Upper Attempts',
            'Dl0bTxVPvyB3xmKRaMEe': 'AUTO Upper Made', 'TPYj5Om3DKLa7b410gD6': 'AUTO Upper Made',
            '6QG3gl5E2RyvYvRYnoNr': 'AUTO Lower Attempts', 'EifnbrfL7ZU9LM8GmkDU': 'AUTO Lower Attempts',
            'advTgK3A354D77txD4YC': 'AUTO Lower Made', 'PvBEDLgUG8DY9dOPWNlx': 'AUTO Lower Made',
            'HqOgHWcmDwHuS5SbkeW9': 'AUTO Comments',
            'ZpWaId8Aq3ryJTpLL3fy': 'TELEOP Upper Attempts', '2LSE7sFI2J4NrKSG3csk': 'TELEOP Upper Attempts', 
            'bMEWpnglaKMAZd3Ds7fP': 'TELEOP Upper Made', 'WahYrpAakxulEWULzpZe': 'TELEOP Upper Made',
            'lCxZegc80cFo4FeYJV02': 'TELEOP Lower Attempts', 'LPbRDIskSTx4zFruY2JX': 'TELEOP Lower Attempts', 
            'L9nU9zioo3j8OgP8ehVa': 'TELEOP Lower Made','Or9Zah9VWm5JQUDfWqCI': 'TELEOP Lower Made',
            'qW9pGE83hpB0uFKvZJTo': 'TELEOP Cargo Cycle Time', 'FHfF2KOElFV9FWnYaFZW': 'TELEOP Hangar Cycle Time', 
            'jRFOomtMICa8dRwmVDE9': 'TELEOP Cargo Shooting Comments','julVFr9AoxLBkp5Nempw': 'TELEOP Cargo Shooting Comments',
            'AbcYYzcqQEJwUUhB4sZ3': 'END GAME Climb Rung','N7FsGJluZUMWOOasXAq5': 'END GAME Climb Rung',
            'jiAEi6v5sNbqvDW1ckAy': 'DEFENSE Rating', 'nLq6ty34RPyqpOixXNQp': 'DEFENSE Rating', 
            'eMyY331WfizHokzqMMJ1': 'DEFENSE Comments','mXQ052v4OBe0VFFODoWN': 'DEFENSE Comments',
            'lG2IkAch1TRPEHI6f5r3': 'PENALTIES Incurred',  'yIeGBuBJ1Cl11ii6kVOb': 'PENALTIES Incurred', 
            'HLhFaaXhRjiL8Cgaq7wp': 'PENALTIES Comments', 'ZZ6jSP6xBWATNEGGuNhs': 'PENALTIES Comments',
            'NytQ2N41UjzEePOwlQSp': 'TELEOP Speed Rating',  'dAgjeXS5NXC8IvTuzmGK': 'TELEOP Speed Rating', 
            'SceDYFvcXPPW1jlR8zP8': 'RELIABILITY Robot Broke','HH1xG4fI6to5xoqZoW0H': 'RELIABILITY Robot Broke',
            'yPHvlaaDtsi0TGWZMTpJ': 'RELIABILITY Robot Stalled', 'uRekeYWKuxJBrmfdAMM9': 'RELIABILITY Robot Stalled', 
            'Ig1WZWJkdGycG4CP24OZ': 'RELIABILITY Comments', '9vC1VbNlcFFAdyArqnjh': 'RELIABILITY Comments',
            '0zLexNpiJlvudyMtgPyP': 'Additional Comments', 'RkwO8ZrwGzGtxa1R47Jo': 'Additional Comments'
           }       

In [6]:
# Index to use for each row of match metrics that scouts have provided
idx_allmetrics = 0

# A pandas DataFrame to collect all the metrics.  
# Each row is a match and we will identify the team that was scouted in the row. 
df_allmetrics = pd.DataFrame()

# We will collect all the metrics for the current match into a dictionary.
# This dictionary will be added to the end of the df_allmetrics dataframe.
dict_matchmetrics = {}
    
# Let's run through all the scouting match metrics data we have from our scouts
for index_team, row_team in df_scouting.iterrows():
    
    # Let's grab all the matches of metrics that we have for the current team
    df_matches = pd.DataFrame.from_dict(row_team['teams'])
    
    # We'll now iterate through each match's metrics for the current team
    for index_match, row_match in df_matches.iterrows():
        
        # Setting the team number and timestamp for the current match's metrics
        team = index_team
        timestamp = pd.Timestamp(row_match['timestamp'], unit='ms')  
        
        # Adding the team number and timestamp into the dictionary that we will add as a row of metrics
        dict_matchmetrics['Index'] = [idx_allmetrics]        
        dict_matchmetrics['Team'] = [team]
        dict_matchmetrics['Timestamp'] = [timestamp.strftime('%m-%d-%Y')]
        
        # When the JSON file was imported, the metrics were still in a set of key, value, category, name set of fields
        # under the 'metrics' column of the dataframe that was created when we read the JSON file.
        # We will iterate through these key, value type sets of data to parse out what we want from the scouting metrics.
        d_metrics = row_match['metrics']
        
        # Using items() + list comprehension
        # Substring Key match in dictionary
        for key, val in d_metrics.items():
            # print("key: ", key, "Type:", val['type'],"Name:", val['name'], "Category:", val['category'],"Value:", val['value'])
            # print("Label will be:", d_labels[key], 'Index:', i)
            
            if key == 'jiAEi6v5sNbqvDW1ckAy':
                val['value'] = int(val['value'])
            
            # Adding metrics to our dictionary that will be added as a row once we have all of them for the match.
            if val['value'] is None:
                dict_matchmetrics[d_labels[key]] = ['']
            elif len(val['name']) < 6:
                dict_matchmetrics[val['name']] = val['value']              
            elif val['type'] in {"boolean", "number","stopwatch"}:
                dict_matchmetrics[d_labels[key]] = [val['value']]                
            else:
                dict_matchmetrics[d_labels[key]] = [val['value']]                
        
        # Let's convert our dictionary of metrics for a match into a DataFrame to be ready to add
        new_matchmetrics = pd.DataFrame(dict_matchmetrics)
        
        # If this is our first row, let's initialize our all metrics DataFrame to this first row.
        if df_allmetrics.empty:
            # If first row let's set it
            df_allmetrics = new_matchmetrics
        else:
            #append new row to the dataframe
            df_allmetrics = pd.concat([df_allmetrics, new_matchmetrics], ignore_index=True)
    
        # Incrementing to be ready to use as an index to each row
        idx_allmetrics = idx_allmetrics + 1

# Let's just see what we have!
display(df_allmetrics)

,Index,Team,Timestamp,Scout,Match,AUTO Taxied,AUTO Upper Attempts,AUTO Upper Made,AUTO Lower Attempts,AUTO Lower Made,...,PENALTIES Comments,RELIABILITY Robot Broke,RELIABILITY Robot Stalled,RELIABILITY Comments,Additional Comments,Name,TELEOP Cargo Cycle Time,TELEOP Hangar Cycle Time,TELEOP Speed Rating,Scouted Match
0,0,1011,03-25-2022,Kp + MH,Q3,False,0,0,0,0,...,,False,False,,Was not there. Very slow. Barely functional,NaN,NaN,NaN,NaN,NaN
1,1,1011,03-25-2022,GM,Q13,True,0,0,0,0,...,,False,False,Started moving very slowly after a bit,,NaN,NaN,NaN,NaN,NaN
2,2,1011,03-25-2022,NG,Q19,False,0,0,0,0,...,,False,False,,,NaN,NaN,NaN,NaN,NaN
3,3,1011,03-25-2022,MN,Q28,True,2,2,0,0,...,,False,False,,,NaN,NaN,NaN,NaN,NaN
4,4,1011,03-25-2022,GM,Q28,True,0,0,0,0,...,,False,True,Kept stopping for long periods of time,,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,424,8283,03-25-2022,CM,Q58,True,0,0,1,1,...,,False,False,,,NG,[],[5881],Slow,
425,425,8334,03-25-2022,NG,Q9,True,1,1,0,0,...,,False,False,,,NG,[],[5881],Slow,
426,426,8334,03-25-2022,CM,Q14,True,0,0,0,0,...,,False,False,,Just drove around at end. No attempt to approa...,NG,[],[5881],Slow,
427,427,8334,03-25-2022,Kg,Qr.28,True,1,1,0,0,...,,False,False,,,NG,[],[5881],Slow,


In [7]:
df_allmetrics['Upper Attempts'] = df_allmetrics['AUTO Upper Attempts'] + df_allmetrics['TELEOP Upper Attempts']
df_allmetrics['Upper Made'] = df_allmetrics['AUTO Upper Made'] + df_allmetrics['TELEOP Upper Made']
df_allmetrics['Upper % Made'] = df_allmetrics['Upper Made'] / df_allmetrics['Upper Attempts'] * 100
df_allmetrics['Lower Attempts'] = df_allmetrics['AUTO Lower Attempts'] + df_allmetrics['TELEOP Lower Attempts']
df_allmetrics['Lower Made'] = df_allmetrics['AUTO Lower Made'] + df_allmetrics['TELEOP Lower Made']
df_allmetrics['Lower % Made'] = df_allmetrics['Lower Made'] / df_allmetrics['Lower Attempts'] * 100

In [8]:
df_groupbyTeam = df_allmetrics.groupby('Team').agg(
    max_upper_attempts=pd.NamedAgg(column='Upper Attempts', aggfunc=max),
    min_upper_attempts=pd.NamedAgg(column='Upper Attempts', aggfunc=min),
    mean_upper_attempts=pd.NamedAgg(column='Upper Attempts', aggfunc='mean'),
    max_upper_pctmade=pd.NamedAgg(column='Upper % Made', aggfunc=max),
    min_upper_pctmade=pd.NamedAgg(column='Upper % Made', aggfunc=min),
    mean_upper_pctmade=pd.NamedAgg(column='Upper % Made', aggfunc='mean'),
    max_lower_attempts=pd.NamedAgg(column='Lower Attempts', aggfunc=max),
    min_lower_attempts=pd.NamedAgg(column='Lower Attempts', aggfunc=min),
    mean_lower_attempts=pd.NamedAgg(column='Lower Attempts', aggfunc='mean'),
    max_lower_pctmade=pd.NamedAgg(column='Lower % Made', aggfunc=max),
    min_lower_pctmade=pd.NamedAgg(column='Lower % Made', aggfunc=min),
    mean_lower_pctmade=pd.NamedAgg(column='Lower % Made', aggfunc='mean'),
    count=pd.NamedAgg(column='Match', aggfunc='count')
)

In [9]:
# Let's see what we've got!
display(df_groupbyTeam)

,max_upper_attempts,min_upper_attempts,mean_upper_attempts,max_upper_pctmade,min_upper_pctmade,mean_upper_pctmade,max_lower_attempts,min_lower_attempts,mean_lower_attempts,max_lower_pctmade,min_lower_pctmade,mean_lower_pctmade,count
Team,,,,,,,,,,,,,
159,10,0,4.272727,100.000000,0.000000,31.000000,1,0,0.181818,0.000000,0.000000,0.000000,11
662,1,0,0.111111,0.000000,0.000000,0.000000,5,0,2.111111,100.000000,0.000000,78.095238,9
1011,12,0,1.555556,91.666667,0.000000,45.833333,4,0,0.555556,75.000000,0.000000,37.500000,9
1245,10,0,2.125000,inf,0.000000,inf,9,0,3.375000,100.000000,0.000000,60.833333,8
1303,0,0,0.000000,NaN,NaN,NaN,1,0,0.222222,0.000000,0.000000,0.000000,9
1332,0,0,0.000000,NaN,NaN,NaN,11,2,7.100000,100.000000,62.500000,91.336580,10
1339,20,0,15.250000,100.000000,0.000000,81.743857,0,0,0.000000,NaN,NaN,NaN,12
1410,0,0,0.000000,NaN,NaN,NaN,12,1,5.666667,100.000000,72.727273,96.969697,9
1619,26,0,15.705882,94.444444,56.250000,76.842664,0,0,0.000000,NaN,NaN,NaN,17


In [10]:
# Let's remove all the '_' underscores from our names
df_groupbyTeam.columns = df_groupbyTeam.columns.str.replace("_", " ")
# Let's remove all the '_' underscores from our names
df_groupbyTeam.columns = df_groupbyTeam.columns.str.replace("pct", "% ")

df_groupbyTeam['Team'] = df_groupbyTeam.index

# Let's see the columns that we have now
df_groupbyTeam.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 159 to 8334
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   max upper attempts   41 non-null     int64  
 1   min upper attempts   41 non-null     int64  
 2   mean upper attempts  41 non-null     float64
 3   max upper % made     32 non-null     float64
 4   min upper % made     32 non-null     float64
 5   mean upper % made    32 non-null     float64
 6   max lower attempts   41 non-null     int64  
 7   min lower attempts   41 non-null     int64  
 8   mean lower attempts  41 non-null     float64
 9   max lower % made     34 non-null     float64
 10  min lower % made     34 non-null     float64
 11  mean lower % made    33 non-null     float64
 12  count                41 non-null     int64  
 13  Team                 41 non-null     int64  
dtypes: float64(8), int64(6)
memory usage: 4.8 KB


In [11]:
# Making sure we have our percentage numbers as a fraction to convert appropriate within Excel
df_groupbyTeam['max upper % made'] = df_groupbyTeam['max upper % made'] / 100
df_groupbyTeam['min upper % made'] = df_groupbyTeam['min upper % made'] / 100
df_groupbyTeam['mean upper % made'] = df_groupbyTeam['mean upper % made'] / 100
df_groupbyTeam['max lower % made'] = df_groupbyTeam['max lower % made'] / 100
df_groupbyTeam['min lower % made'] = df_groupbyTeam['min lower % made'] / 100
df_groupbyTeam['mean lower % made'] = df_groupbyTeam['mean lower % made'] / 100

## Output the full list of Match Metrics we have from the Scouts

In [12]:
#  All match metrics exporting to a CSV file to be used by other scripts or to look at AS-IS
df_allmetrics[['Team','Timestamp','Name','Match','Upper Attempts','Upper % Made','AUTO Upper Attempts','AUTO Upper Made','TELEOP Upper Attempts','TELEOP Upper Made','Lower Attempts','Lower % Made','AUTO Lower Attempts','AUTO Lower Made','TELEOP Lower Attempts','TELEOP Lower Made','TELEOP Cargo Shooting Comments','AUTO Taxied','END GAME Climb Rung','TELEOP Speed Rating','DEFENSE Rating','DEFENSE Comments','PENALTIES Incurred','PENALTIES Comments','RELIABILITY Robot Broke','RELIABILITY Robot Stalled','RELIABILITY Comments','Additional Comments']].to_csv("LatestResults/AllMatchMetrics.csv", index=False, float_format="%.1f")

In [13]:
#  The summary[max, min, mean] of match metrics by team exporting to a CSV file to be used by other scripts or to look at AS-IS
df_groupbyTeam[['Team','count','mean upper attempts','mean upper % made','mean lower attempts','mean lower % made','max upper attempts','min upper attempts','max upper % made','min upper % made','max lower attempts','min lower attempts','max lower % made','min lower % made']].to_csv("LatestResults/MaxMinMeanOfTeams.csv", index=False, float_format="%.1f")


In [14]:
# Attempting to write to Excel to look at AS-IS 
filename_Excel = "LatestResults/OurScouting.xlsx"
with pd.ExcelWriter(filename_Excel, mode='a', if_sheet_exists='overlay', datetime_format='mmm d yyyy hh:mm:ss', date_format='hh:mm:ss.000', engine="openpyxl") as writer:
    df_groupbyTeam[['Team','count','mean upper attempts','mean upper % made','mean lower attempts','mean lower % made','max upper attempts','min upper attempts','max upper % made','min upper % made','max lower attempts','min lower attempts','max lower % made','min lower % made']].to_excel(writer, sheet_name="MaxMinMean of Teams", index=False, float_format="%.1f")
    df_allmetrics[['Team','Timestamp','Name','Match','Upper Attempts','Upper % Made','Lower Attempts','Lower % Made','AUTO Upper Attempts','AUTO Upper Made','AUTO Lower Attempts','AUTO Lower Made','TELEOP Upper Attempts','TELEOP Upper Made','TELEOP Lower Attempts','TELEOP Lower Made','TELEOP Cargo Shooting Comments','AUTO Taxied','END GAME Climb Rung','TELEOP Speed Rating']].to_excel(writer, sheet_name="Shooting Offensive", index=False, float_format="%.1f")
    df_allmetrics[['Team','Timestamp','Name','Match','AUTO Taxied','END GAME Climb Rung','TELEOP Speed Rating']].to_excel(writer, sheet_name="Other Offensive", index=False, float_format="%.1f")
    df_allmetrics[['Team','Timestamp','Name','Match','DEFENSE Rating','DEFENSE Comments','PENALTIES Incurred','PENALTIES Comments']].to_excel(writer, sheet_name="Defense", index=False, float_format="%.1f")
    df_allmetrics[['Team','Timestamp','Name','Match','RELIABILITY Robot Broke','RELIABILITY Robot Stalled','RELIABILITY Comments','Additional Comments']].to_excel(writer, sheet_name="Reliability", index=False, float_format="%.1f")
    df_allmetrics.to_excel(writer, sheet_name="All Data", index=True, float_format="%.1f")


## Next steps are for the last day of our event to focus down on the teams of interest for Alliance Selections

In [15]:
# Let's read in a simple list of team numbers that we are interested in.  Each team is one line
df_teamsofinterest = pd.read_csv('LatestInputOfScoutData/teamsofinterest.csv')

# Let's read in the scout Match Metrics from Robot Scouter that have a timestamp of today or the last few matches the afternoon before
df_todaysmetrics = pd.read_csv('LatestInputOfScoutData/OurScouting-TodaySince1030AM.csv')
display(df_todaysmetrics)

,Index,Team,Timestamp,Scout,Match,AUTO Taxied,AUTO Upper Attempts,AUTO Upper Made,AUTO Lower Attempts,AUTO Lower Made,...,TELEOP Cargo Cycle Time,TELEOP Hangar Cycle Time,TELEOP Speed Rating,Scouted Match,Upper Attempts,Upper Made,Upper % Made,Lower Attempts,Lower Made,Lower % Made
0,110,1977,03-26-2022,Kg,Qr.62,True,2,0,0,0,...,[],[],Unknown,NaN,2,0,0.0,2,2,100.0
1,35,1332,03-26-2022,Kg,Q74,True,0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,NaN,6,5,83.3
2,84,1619,03-26-2022,GM,Q74,True,3,1,0,0,...,[],[],Unknown,NaN,20,17,85.0,0,0,NaN
3,201,3200,03-26-2022,NG,Q74,True,1,1,0,0,...,"[23026, 12739, 17077, 11397, 13215, 11611, 14169]",[5217],Average,NaN,7,7,100.0,0,0,NaN
4,417,8250,03-26-2022,DD,Q74,True,1,1,0,0,...,[],[5881],Slow,NaN,3,2,66.7,0,0,NaN
5,184,2996,03-26-2022,NG,Q73,True,1,1,0,0,...,[],[],Average,NaN,11,6,54.5,0,0,NaN
6,47,1339,03-26-2022,GM,Q72,True,4,2,0,0,...,[9386],[28989],Average,NaN,19,15,78.9,0,0,NaN
7,67,159,03-26-2022,DD,Q72,True,2,0,0,0,...,[],[],Average,NaN,6,0,0.0,0,0,NaN
8,298,4499,03-26-2022,Kg,Q72,True,1,0,0,0,...,[],[],Fast,NaN,6,3,50.0,0,0,NaN
9,408,7737,03-26-2022,MB,Q72,True,0,0,0,0,...,[],[5881],Slow,NaN,0,0,NaN,0,0,NaN


In [16]:
#  Let's get some averages for these teams of interest based on the last set of data we have collected on them in
#  the Robot Scouter MATCH METRICS template
df_groupbyTeam = df_todaysmetrics.groupby('Team').agg(
    max_upper_attempts=pd.NamedAgg(column='Upper Attempts', aggfunc=max),
    min_upper_attempts=pd.NamedAgg(column='Upper Attempts', aggfunc=min),
    mean_upper_attempts=pd.NamedAgg(column='Upper Attempts', aggfunc='mean'),
    max_upper_pctmade=pd.NamedAgg(column='Upper % Made', aggfunc=max),
    min_upper_pctmade=pd.NamedAgg(column='Upper % Made', aggfunc=min),
    mean_upper_pctmade=pd.NamedAgg(column='Upper % Made', aggfunc='mean'),
    max_lower_attempts=pd.NamedAgg(column='Lower Attempts', aggfunc=max),
    min_lower_attempts=pd.NamedAgg(column='Lower Attempts', aggfunc=min),
    mean_lower_attempts=pd.NamedAgg(column='Lower Attempts', aggfunc='mean'),
    max_lower_pctmade=pd.NamedAgg(column='Lower % Made', aggfunc=max),
    min_lower_pctmade=pd.NamedAgg(column='Lower % Made', aggfunc=min),
    mean_lower_pctmade=pd.NamedAgg(column='Lower % Made', aggfunc='mean'),
    count=pd.NamedAgg(column='Match', aggfunc='count')
)

In [17]:
# Let's remove all the '_' underscores from our names
df_groupbyTeam.columns = df_groupbyTeam.columns.str.replace("_", " ")
# Let's remove all the '_' underscores from our names
df_groupbyTeam.columns = df_groupbyTeam.columns.str.replace("pct", "% ")

df_groupbyTeam['Team'] = df_groupbyTeam.index

In [18]:
# Making sure we have our percentage numbers as a fraction to convert appropriate within Excel
df_groupbyTeam['max upper % made'] = df_groupbyTeam['max upper % made'] / 100
df_groupbyTeam['min upper % made'] = df_groupbyTeam['min upper % made'] / 100
df_groupbyTeam['mean upper % made'] = df_groupbyTeam['mean upper % made'] / 100
df_groupbyTeam['max lower % made'] = df_groupbyTeam['max lower % made'] / 100
df_groupbyTeam['min lower % made'] = df_groupbyTeam['min lower % made'] / 100
df_groupbyTeam['mean lower % made'] = df_groupbyTeam['mean lower % made'] / 100

## Output the latest stats for our teams of interest for Alliance Selections

In [19]:
#  All match metrics exporting to a CSV file to be used by other scripts or to look at AS-IS
df_todaysmetrics[['Team','Timestamp','Name','Match','Upper Attempts','Upper % Made','AUTO Upper Attempts','AUTO Upper Made','TELEOP Upper Attempts','TELEOP Upper Made','Lower Attempts','Lower % Made','AUTO Lower Attempts','AUTO Lower Made','TELEOP Lower Attempts','TELEOP Lower Made','TELEOP Cargo Shooting Comments','AUTO Taxied','END GAME Climb Rung','TELEOP Speed Rating','DEFENSE Rating','DEFENSE Comments','PENALTIES Incurred','PENALTIES Comments','RELIABILITY Robot Broke','RELIABILITY Robot Stalled','RELIABILITY Comments','Additional Comments']].to_csv("LatestResults/Todays_AllMatchMetrics.csv", index=False, float_format="%.1f")

In [20]:
#  The summary[max, min, mean] of match metrics by team exporting to a CSV file to be used by other scripts or to look at AS-IS
df_groupbyTeam[['Team','count','mean upper attempts','mean upper % made','mean lower attempts','mean lower % made','max upper attempts','min upper attempts','max upper % made','min upper % made','max lower attempts','min lower attempts','max lower % made','min lower % made']].to_csv("LatestResults/Todays_MaxMinMeanOfTeams.csv", index=False, float_format="%.1f")


In [21]:
# Attempting to write to Excel
filename_Excel = "LatestResults/Todays_OurScouting.xlsx"
with pd.ExcelWriter(filename_Excel, mode='a', if_sheet_exists='overlay', datetime_format='mmm d yyyy hh:mm:ss', date_format='hh:mm:ss.000', engine="openpyxl") as writer:
    df_groupbyTeam[['Team','count','mean upper attempts','mean upper % made','mean lower attempts','mean lower % made','max upper attempts','min upper attempts','max upper % made','min upper % made','max lower attempts','min lower attempts','max lower % made','min lower % made']].to_excel(writer, sheet_name="MaxMinMean of Teams", index=False, float_format="%.1f")
    df_allmetrics[['Team','Timestamp','Name','Match','Upper Attempts','Upper % Made','Lower Attempts','Lower % Made','AUTO Upper Attempts','AUTO Upper Made','AUTO Lower Attempts','AUTO Lower Made','TELEOP Upper Attempts','TELEOP Upper Made','TELEOP Lower Attempts','TELEOP Lower Made','TELEOP Cargo Shooting Comments','AUTO Taxied','END GAME Climb Rung','TELEOP Speed Rating']].to_excel(writer, sheet_name="Shooting Offensive", index=False, float_format="%.1f")
    df_allmetrics[['Team','Timestamp','Name','Match','AUTO Taxied','END GAME Climb Rung','TELEOP Speed Rating']].to_excel(writer, sheet_name="Other Offensive", index=False, float_format="%.1f")
    df_allmetrics[['Team','Timestamp','Name','Match','DEFENSE Rating','DEFENSE Comments','PENALTIES Incurred','PENALTIES Comments']].to_excel(writer, sheet_name="Defense", index=False, float_format="%.1f")
    df_allmetrics[['Team','Timestamp','Name','Match','RELIABILITY Robot Broke','RELIABILITY Robot Stalled','RELIABILITY Comments','Additional Comments']].to_excel(writer, sheet_name="Reliability", index=False, float_format="%.1f")
    df_allmetrics.to_excel(writer, sheet_name="All Data", index=True, float_format="%.1f")
